# 6  Data Loading, Storage, and File Formats

Reading data and making it accessible (often called data loading) is a necessary first step for using most of the tools in this book. The term ```parsing``` is also sometimes used to describe loading text data and interpreting it as tables and different data types. I’m going to focus on data input and output using pandas, though there are numerous tools in other libraries to help with reading and writing data in various formats.

Input and output typically fall into a few main categories: reading text files and other more efficient on-disk formats, loading data from databases, and interacting with network sources like web APIs.

## 6.1 Reading and Writing Data in Text Format

**Indexing**
Can treat one or more columns as the returned DataFrame, and whether to get column names from the file, arguments you provide, or not at all.

**Type inference and data conversion**
Includes the user-defined value conversions and custom list of missing value markers.

**Date and time parsing**
Includes a combining capability, including combining date and time information spread over multiple columns into a single column in the result.

**Iterating**
Support for iterating over chunks of very large files.

**Unclean data issues**
Includes skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

Unix **cat** shell command to print the raw contents of the file to the screen.

In [1]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("examples/ex1.csv")

In [4]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [6]:
pd.read_csv("examples/ex2.csv",header=None)


,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv("examples/ex2.csv", names = ["a", "b","c","d"])

,a,b,c,d
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [8]:
names = ["a", "b", "c", "d", "message"]

In [9]:
pd.read_csv("examples/ex2.csv", names = names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [11]:
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1","key2"])

In [12]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [13]:
!cat examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [14]:
result = pd.read_csv("examples/ex3.txt", sep="\s+")

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/var/folders/5q/z1l12s4n7y1bxgmlhw1vkr7c0000gp/T/ipykernel_34919/3778575481.py:1: SyntaxWarning: invalid escape sequence '\s'
  result = pd.read_csv("examples/ex3.txt", sep="\s+")


In [15]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [16]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [17]:
pd.read_csv("examples/ex4.csv", skiprows =[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [18]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [19]:
result=pd.read_csv("examples/ex5.csv")

In [20]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [21]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [22]:
result = pd.read_csv("examples/ex5.csv", na_values=["NULL"])

In [23]:
result2 = pd.read_csv("examples/ex5.csv", keep_default_na=False)

In [24]:
result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [25]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [26]:
result3 = pd.read_csv("examples/ex5.csv", keep_default_na=False, na_values=["NA"])

In [27]:
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [28]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [29]:
pd.read_csv("examples/ex5.csv", na_values=sentinels, keep_default_na=False)

NameError: name 'sentinels' is not defined

## Reading Text Files in Pieces

In [ ]:
pd.options.display.max_rows = 10

In [ ]:
result =pd.read_csv("examples/ex6.csv")

In [ ]:
result

In [ ]:
chunker = pd.read_csv("examples/ex6.csv", chunksize = 1000)

In [ ]:
type(chunker)

In [ ]:
pd.read_csv("examples/ex6.csv", chunksize = 1000)
tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

In [ ]:
tot[:10]

## Writing Data to Text Format

In [ ]:
data = pd.read_csv("examples/ex5.csv")

In [ ]:
data

In [ ]:
data.to_csv("examples/out.csv")

In [ ]:
!cat examples/out.csv

In [ ]:
import sys

In [ ]:
data.to_csv(sys.stdout, sep="|")

In [ ]:
data.to_csv(sys.stdout, na_rep="NULL")

In [ ]:
data.to_csv(sys.stdout, index=False, header=False)

In [ ]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

## Working with Other Delimited Formats

In [ ]:
!cat examples/ex7.csv

In [ ]:
import csv

In [ ]:
f = open("examples/ex7.csv")

In [ ]:
reader = csv.reader(f)

In [ ]:
for line in reader: print(line)

In [ ]:
f.close()

In [ ]:
with open("examples/ex7.csv") as f: lines =list(csv.reader(f))

In [ ]:
header, values = lines[0], lines[1:]

In [ ]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [ ]:
data_dict

In [ ]:
class my_dialect(csv.Dialect): 
    lineterminator = '\n"
    delimiter = ";"
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [ ]:
reader = csv.reader(f, dialect=my_dialect)

In [ ]:
reader = csv.reader(f, delimiter="|")

In [ ]:
with open("mydata.csv", "w") as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(("one", "two", "three"))
    writer.writerow(("1", "2", "3"))
    writer.writerow(("4","5","6"))
    writer.writerow(("7", "8","9"))

## JSON Data


In [ ]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

In [ ]:
import json

In [ ]:
result = json.loads(obj)

In [ ]:
result

In [ ]:
asjson =json.dumps(result)

In [ ]:
asjson

In [ ]:
siblings =pd.DataFrame(result["siblings"], columns=["name", "age"])

In [ ]:
siblings

In [ ]:
!cat examples/example.json

In [ ]:
data =pd.read_json("examples/example.json")

In [ ]:
data

In [ ]:
data.to_json(sys.stdout)

## XML and HTML: Web Scraping

In [ ]:
pip install lxml

In [ ]:
tables = pd.read_html("examples/fdic_failed_bank_list.html")

In [ ]:
len(tables)

In [ ]:
failures = tables[0]

In [ ]:
failures.head()

In [ ]:
close_timestamps =pd.to_datetime(failures["Closing Date"])

In [ ]:
close_timestamps.dt.year.value_counts()

### Parsing XML with lxml.objectify

In [ ]:
from lxml import objectify

In [ ]:
path = "datasets/mta_perf/Performance_MNR.xml"

In [ ]:
with open(path) as f: parsed = objectify.parse(f)

In [ ]:
root = parsed.getroot()

In [ ]:
data = []

In [ ]:
skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]
for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [ ]:
perf = pd.DataFrame(data)
perf.head()

In [ ]:
perf = pd.DataFrame(data)

In [ ]:
perf2 = pd.read_xml(path)

In [ ]:
perf2.head()

## 6.2 Binary Data Formats

In [ ]:
frame = pd.read_csv("examples/ex1.csv")

In [ ]:
frame

In [ ]:
frame.to_pickle("examples/frame_pickle")

In [ ]:
pd.read_pickle("examples/frame_pickle")

For example, if you install the pyarrow package

In [ ]:
pip install pyarrow

In [ ]:
fec = pd.read_parquet('datasets/fec/fec.parquet')

## Reading Microsoft Excel Files

In [ ]:
pip install openpyxl xlrd

In [ ]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")

In [ ]:
xlsx.sheet_names

In [ ]:
xlsx.parse(sheet_name="Sheet1")

In [ ]:
xlsx.parse(sheet_name="Sheet1", index_col=0)

In [ ]:
frame = pd.read_excel("examples/ex1.xlsx", sheet_name ="Sheet1")

In [ ]:
writer = pd.ExcelWriter("examples/ex2.xlsx")

In [ ]:
frame.to_excel(writer, "Sheet1")

In [ ]:
writer.close()

### Using HDF5 Format

In [ ]:
pip install pytables

In [ ]:
import numpy as np

In [ ]:
frame = pd.DataFrame({"a": np.random.standard_normal(100)})

In [ ]:
store = pd.HDFStore("examples/mydata.h5")

In [30]:
store["obj1"] = frame

UsageError: Unknown variable '[obj1]'


In [31]:
store["obj1_col"] = frame["a"]

UsageError: Unknown variable '[obj1_col]'


In [32]:
store

AttributeError: 'PickleShareDB' object has no attribute 'keys'

In [ ]:
store["obj1"]

In [ ]:
store.put("obj2", frame, format="table")

In [ ]:
store.select("obj2", where=["index >= 1- and index <= 15"])

In [ ]:
store.close()

In [ ]:
frame.to_hdf("examples/mydata.h5", "obj3", format="table")

In [33]:
pd.read_hdf("examples/mydata.h5", "obj3", where=["index < 5"])

FileNotFoundError: File examples/mydata.h5 does not exist

In [34]:
import os

In [35]:
os.remove("examples/mydata.h5")

FileNotFoundError: [Errno 2] No such file or directory: 'examples/mydata.h5'

# 6.3 Interacting with Web APIs

In [36]:
pip install requests


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.3/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [38]:
import requests

In [39]:
url = "https://api.github.com/repos/pandas-dev/pandas/issues"

In [40]:
resp = requests.get(url)

In [41]:
resp.raise_for_status()

In [42]:
resp

<Response [200]>

In [43]:
data = resp.json()

In [44]:
data[0]["title"]

'BUG Fix for Add numeric_only to function signature of DataFrameGroupBy.cumprod and `DataFrameGroupBy.cumsum '

In [45]:
issues = pd.DataFrame(data, columns = ["number", "title","labels", "state"])

In [46]:
issues

,number,title,labels,state
0,59427,BUG Fix for Add numeric_only to function signa...,[],open
1,59425,DOC: Improve DataFrameGroupBy.std ddof doc,[],open
2,59422,BUG: UnboundLocalError when full outer merging...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
3,59421,BUG: merging DataFrames on a column containing...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
4,59419,[pre-commit.ci] pre-commit autoupdate,[],open
5,59418,BUG: Series.gt (and other comparison methods) ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
6,59417,Unable to build pandas from source on Windows,"[{'id': 129350, 'node_id': 'MDU6TGFiZWwxMjkzNT...",open
7,59414,API/TST: expand tests for string any/all reduc...,"[{'id': 35818298, 'node_id': 'MDU6TGFiZWwzNTgx...",open
8,59412,DOC: Improve sort parameter doc of DataFrameGr...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
9,59410,Use bitmask for MaskedArray mask,[],open


# 6.4 Interacting with Databases

In [48]:
import sqlite3

In [53]:
query = """CREATE TABLE test (a VARCHAR(20), b VARCHAR(20), c REAL, d INTEGER);"""

In [54]:
con = sqlite3.connect("mydata.sqlite")

In [55]:
con.execute(query)

In [56]:
con.commit()

In [57]:
data = [("Atlanta", "Georgia", 1.25, 6),("Tallahassee", "Florida", 2.6, 3),
("Sacramento", "California", 1.7, 5)]

In [58]:
stmt = "INSERT INTO test VALUES(?,?,?,?)"

In [59]:
con.executemany(stmt, data)

In [60]:
con.commit()

In [61]:
cursor = con.execute("SELECT * FROM test")

In [62]:
rows = cursor.fetchall()

In [63]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [64]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [65]:
pd.DataFrame(rows, columns = [x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [66]:
pip install sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.3/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [68]:
import sqlalchemy as sqla

In [69]:
db = sqla.create_engine("sqlite:///mydata.sqlite")

In [71]:
pd.read_sql("SELECT * FROM test", db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
